In [1]:
# SG by category model 
using Pkg, JuMP, Ipopt, DataFrames, CSV, Random, Distributions

In [6]:
#pweight_df = CSV.read("C:/Users/Luke/OneDrive/Research/sg_data/realv2_plyr_weights.csv") # nodsr weights
#cweight_df = CSV.read("C:/Users/Luke/OneDrive/Research/sg_data/realv2_course_weights.csv")


pweight_df = CSV.read("C:/Users/Luke/OneDrive/Research/sg_data/realv2_scaled_round_sg_plyr_weights.csv") # dsr weights
cweight_df = CSV.read("C:/Users/Luke/OneDrive/Research/sg_data/realv2_scaled_round_sg_course_weights.csv")

real_data = CSV.read("C:/Users/Luke/OneDrive/Research/sg_data/scaled_round_sg_2018.csv") # scaled data

real_data[!, :SG_total] = real_data[!, :SG_appr] + real_data[!, :SG_short]  +real_data[!, :SG_drive] + real_data[!, :SG_putt] 
n_real = length(real_data[!,:player])
num_plyrs = length(unique(real_data[!,:player]))
num_courses = length(unique(real_data[!,:course_name]))

37

# NO DSR 

In [5]:
mod_real = Model(with_optimizer(Ipopt.Optimizer))

@variable(mod_real, p_sg_drive[1:num_plyrs])
@variable(mod_real, p_sg_appr[1:num_plyrs])
@variable(mod_real, p_sg_short[1:num_plyrs])
@variable(mod_real, p_sg_putt[1:num_plyrs])

@variable(mod_real, c_sg_drive[1:num_courses])
@variable(mod_real, c_sg_appr[1:num_courses])
@variable(mod_real, c_sg_short[1:num_courses])
@variable(mod_real, c_sg_putt[1:num_courses])

p_sg_drive_ = [p_sg_drive[i+1] for i in real_data[!,:p_idx]]
p_sg_appr_ = [p_sg_appr[i+1] for i in real_data[!,:p_idx]]
p_sg_short_ = [p_sg_short[i+1] for i in real_data[!,:p_idx]]
p_sg_putt_ = [p_sg_putt[i+1] for i in real_data[!,:p_idx]]

c_sg_drive_ = [c_sg_drive[i+1] for i in real_data[!,:c_idx]]
c_sg_appr_ = [c_sg_appr[i+1] for i in real_data[!,:c_idx]]
c_sg_short_ = [c_sg_short[i+1] for i in real_data[!,:c_idx]]
c_sg_putt_ = [c_sg_putt[i+1] for i in real_data[!,:c_idx]]

hsg_wmean_drive = @expression(mod_real, sum(c_sg_drive[i]*(cweight_df[!,:weight][i]) for i in 1:num_courses))
hsg_wmean_appr = @expression(mod_real, sum(c_sg_appr[i]*(cweight_df[!,:weight][i]) for i in 1:num_courses))
hsg_wmean_short = @expression(mod_real, sum(c_sg_short[i]*(cweight_df[!,:weight][i]) for i in 1:num_courses))
hsg_wmean_putt = @expression(mod_real, sum(c_sg_putt[i]*(cweight_df[!,:weight][i]) for i in 1:num_courses))

hsg_drive_avg_constr = @constraint(mod_real,  hsg_wmean_drive == 0.0) 
hsg_appr_avg_constr = @constraint(mod_real,  hsg_wmean_appr == 0.0) 
hsg_short_avg_constr = @constraint(mod_real,  hsg_wmean_short == 0.0) 
hsg_putt_avg_constr = @constraint(mod_real,  hsg_wmean_putt == 0.0)

psg_wmean_drive = @expression(mod_real, sum(p_sg_drive[i]*(pweight_df[!,:weight][i]) for i in 1:num_plyrs))
psg_wmean_appr = @expression(mod_real, sum(p_sg_appr[i]*(pweight_df[!,:weight][i]) for i in 1:num_plyrs))
psg_wmean_short = @expression(mod_real, sum(p_sg_short[i]*(pweight_df[!,:weight][i]) for i in 1:num_plyrs))
psg_wmean_putt = @expression(mod_real, sum(p_sg_putt[i]*(pweight_df[!,:weight][i]) for i in 1:num_plyrs))

psg_drive_avg_constr = @constraint(mod_real,  psg_wmean_drive == 0.0) 
psg_appr_avg_constr = @constraint(mod_real,  psg_wmean_appr == 0.0) 
psg_short_avg_constr = @constraint(mod_real,  psg_wmean_short == 0.0) 
psg_putt_avg_constr = @constraint(mod_real,  psg_wmean_putt == 0.0)

sg_drive_constr = @constraint(mod_real, [i=1:num_courses], -5 <= c_sg_drive[i] <= 5) 
sg_appr_constr = @constraint(mod_real, [i=1:num_courses],  -5 <= c_sg_appr[i] <= 5) 
sg_short_constr = @constraint(mod_real, [i=1:num_courses],  -5 <= c_sg_short[i] <= 5) 
sg_putt_constr = @constraint(mod_real, [i=1:num_courses],  -5 <= c_sg_putt[i] <= 5) 

psg_drive_constr = @constraint(mod_real, [i=1:num_plyrs], -5 <= p_sg_drive[i] <= 5) 
psg_appr_constr = @constraint(mod_real, [i=1:num_plyrs],  -5 <= p_sg_appr[i] <= 5) 
psg_short_constr = @constraint(mod_real, [i=1:num_plyrs],  -5 <= p_sg_short[i] <= 5) 
psg_putt_constr = @constraint(mod_real, [i=1:num_plyrs],  -5 <= p_sg_putt[i] <= 5) 


@NLobjective(mod_real, Min, sum( (p_sg_drive_[i]*p_sg_drive_[i] - 2*p_sg_drive_[i]*c_sg_drive_[i] + c_sg_drive_[i]*c_sg_drive_[i] - 2*p_sg_drive_[i]*real_data[!,:SG_drive][i] + 2*c_sg_drive_[i]*real_data[!,:SG_drive][i] + real_data[!,:SG_drive][i]*real_data[!,:SG_drive][i]) + (p_sg_appr_[i]*p_sg_appr_[i] - 2*p_sg_appr_[i]*c_sg_appr_[i] + c_sg_appr_[i]*c_sg_appr_[i] - 2*p_sg_appr_[i]*real_data[!,:SG_appr][i] + 2*c_sg_appr_[i]*real_data[!,:SG_appr][i] + real_data[!,:SG_appr][i]*real_data[!,:SG_appr][i]) + (p_sg_short_[i]*p_sg_short_[i] - 2*p_sg_short_[i]*c_sg_short_[i] + c_sg_short_[i]*c_sg_short_[i] - 2*p_sg_short_[i]*real_data[!,:SG_short][i] + 2*c_sg_short_[i]*real_data[!,:SG_short][i] + real_data[!,:SG_short][i]*real_data[!,:SG_short][i]) + (p_sg_putt_[i]*p_sg_putt_[i] - 2*p_sg_putt_[i]*c_sg_putt_[i] + c_sg_putt_[i]*c_sg_putt_[i] - 2*p_sg_putt_[i]*real_data[!,:SG_putt][i] + 2*c_sg_putt_[i]*real_data[!,:SG_putt][i] + real_data[!,:SG_putt][i]*real_data[!,:SG_putt][i]) for i in 1:n_real)/(n_real))

status_real = optimize!(mod_real)

c_idx_df = DataFrame(idx = unique(real_data[!,:c_idx]), course_name = unique(real_data[!,:course_name]))
p_idx_df = DataFrame(idx = unique(real_data[!,:p_idx]), player = unique(real_data[!,:player]))

p_d_label = [value.(p_sg_drive)[i+1] for i in p_idx_df[!,:idx]]
p_a_label = [value.(p_sg_appr)[i+1] for i in p_idx_df[!,:idx]]
p_s_label = [value.(p_sg_short)[i+1] for i in p_idx_df[!,:idx]]
p_p_label = [value.(p_sg_putt)[i+1] for i in p_idx_df[!,:idx]]

c_d_label = [value.(c_sg_drive)[i+1] for i in c_idx_df[!,:idx]]
c_a_label = [value.(c_sg_appr)[i+1] for i in c_idx_df[!,:idx]]
c_s_label = [value.(c_sg_short)[i+1] for i in c_idx_df[!,:idx]]
c_p_label = [value.(c_sg_putt)[i+1] for i in c_idx_df[!,:idx]]


p_rounds = [pweight_df[!,:weight][i+1]*n_real for i in p_idx_df[!,:idx]]
c_rounds = [cweight_df[!,:weight][i+1]*n_real for i in c_idx_df[!,:idx]]
nodsr_obj_val = DataFrame(obj = [sqrt(objective_value(mod_real))])

nodsr_c_sg_final_df = DataFrame(idx = c_idx_df[!,:idx], course_name = c_idx_df[!,:course_name], sg_drive = c_d_label, sg_appr = c_a_label, sg_short = c_s_label, sg_putt = c_p_label,num_rounds = c_rounds)

nodsr_p_final_df = DataFrame(idx = p_idx_df[!,:idx], player = p_idx_df[!,:player], sg_drive = p_d_label, sg_appr = p_a_label, sg_short = p_s_label, sg_putt = p_p_label, num_rounds = p_rounds)


CSV.write("C:/Users/Luke/OneDrive/Research/sg_data/realv2_plyr_nodsr.csv", nodsr_p_final_df)
CSV.write("C:/Users/Luke/OneDrive/Research/sg_data/realv2_course_nodsr.csv", nodsr_c_sg_final_df)
CSV.write("C:/Users/Luke/OneDrive/Research/sg_data/realv2_obj_nodsr.csv", nodsr_obj_val)

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     2164
Number of nonzeros in inequality constraint Jacobian.:     4328
Number of nonzeros in Lagrangian Hessian.............:    21440

Total number of variables............................:     2164
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        8
Total number of inequality constraints...............:     4328
        inequality constraints with only lower bounds:     2164
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:     2164

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

"C:/Users/Luke/OneDrive/Research/sg_data/realv2_obj_nodsr.csv"

#  DSR

In [7]:
## have to run preprocessing to get weights from no dsr first
### WORKING DSR MODEL: 08-24-2020

mod_real = Model(with_optimizer(Ipopt.Optimizer))

@variable(mod_real, p_sg_drive[1:num_plyrs])
@variable(mod_real, c_sg_drive[1:num_courses])
@variable(mod_real, c_dsr_drive[1:num_courses])
@variable(mod_real, p_sg_appr[1:num_plyrs])
@variable(mod_real, c_sg_appr[1:num_courses])
@variable(mod_real, c_dsr_appr[1:num_courses])
@variable(mod_real, p_sg_short[1:num_plyrs])
@variable(mod_real, c_sg_short[1:num_courses])
@variable(mod_real, c_dsr_short[1:num_courses])
@variable(mod_real, p_sg_putt[1:num_plyrs])
@variable(mod_real, c_sg_putt[1:num_courses])
@variable(mod_real, c_dsr_putt[1:num_courses])

p_sg_drive_ = [p_sg_drive[i+1] for i in real_data[!,:p_idx]]
c_sg_drive_ = [c_sg_drive[i+1] for i in real_data[!,:c_idx]]
c_dsr_drive_ = [c_dsr_drive[i+1] for i in real_data[!,:c_idx]]
p_sg_appr_ = [p_sg_appr[i+1] for i in real_data[!,:p_idx]]
c_sg_appr_ = [c_sg_appr[i+1] for i in real_data[!,:c_idx]]
c_dsr_appr_ = [c_dsr_appr[i+1] for i in real_data[!,:c_idx]]
p_sg_short_ = [p_sg_short[i+1] for i in real_data[!,:p_idx]]
c_sg_short_ = [c_sg_short[i+1] for i in real_data[!,:c_idx]]
c_dsr_short_ = [c_dsr_short[i+1] for i in real_data[!,:c_idx]]
p_sg_putt_ = [p_sg_putt[i+1] for i in real_data[!,:p_idx]]
c_sg_putt_ = [c_sg_putt[i+1] for i in real_data[!,:c_idx]]
c_dsr_putt_ = [c_dsr_putt[i+1] for i in real_data[!,:c_idx]]


#dsr_wmean_drive = @expression(mod_real, sum(c_dsr_drive[i]*(filter(row -> (row.c_idx == i-1) , cweight_df)[!,:weight][1]) for i in 1:num_courses))
#dsr_drive_avg_constr = @constraint(mod_real,  dsr_wmean_drive == 0.5) 


hsg_wmean_drive = @expression(mod_real, sum(c_sg_drive[i]*(filter(row -> (row.c_idx == i-1) , cweight_df)[!,:weight][1]) for i in 1:num_courses))
hsg_drive_avg_constr = @constraint(mod_real,  hsg_wmean_drive == 0.0) 

#real_wvar_drive = @NLexpression(mod_real, (num_courses/(num_courses-1))*sum(((filter(row -> (row.c_idx == i-1) , cweight_df)[!,:weight][1])*(abs(c_sg_drive[i]))*(abs(c_sg_drive[i]))) for i in 1:num_courses)) # weighted variance
#real_drive_var_constr = @NLconstraint(mod_real,  real_wvar_drive == 1.0)

psg_wmean_drive = @expression(mod_real, sum(p_sg_drive[i]*(filter(row -> (row.p_idx == i-1) , pweight_df)[!,:weight][1]) for i in 1:num_plyrs))
psg_drive_avg_constr = @constraint(mod_real,  psg_wmean_drive == 0.0) 

real_pwvar_drive = @NLexpression(mod_real, (num_plyrs/(num_plyrs-1))*sum(((filter(row -> (row.p_idx == i-1) , pweight_df)[!,:weight][1])*(abs(p_sg_drive[i]))*(abs(p_sg_drive[i]))) for i in 1:num_plyrs)) # weighted variance
real_pdrive_var_constr = @NLconstraint(mod_real,  real_pwvar_drive == 1.0)

dsr_drive_constr = @constraint(mod_real, [i=1:num_courses], 0 <= c_dsr_drive[i] <= 1) 

sg_drive_apx_constr = @constraint(mod_real, [i=1:num_courses], filter(row -> (row.c_idx == i-1) , cweight_df)[!,:sg_drive_mean][1] -3 <= c_sg_drive[i] <= filter(row -> (row.c_idx == i-1) , cweight_df)[!,:sg_drive_mean][1] + 3) 
psg_drive_apx_constr = @constraint(mod_real, [i=1:num_plyrs], filter(row -> (row.p_idx == i-1) , pweight_df)[!,:sg_drive_mean][1]-3 <= p_sg_drive[i] <= filter(row -> (row.p_idx == i-1) , pweight_df)[!,:sg_drive_mean][1]+3)

#dsr_wmean_appr = @expression(mod_real, sum(c_dsr_appr[i]*(filter(row -> (row.c_idx == i-1) , cweight_df)[!,:weight][1]) for i in 1:num_courses))
#dsr_appr_avg_constr = @constraint(mod_real,  dsr_wmean_appr == 0.5) 

hsg_wmean_appr = @expression(mod_real, sum(c_sg_appr[i]*(filter(row -> (row.c_idx == i-1) , cweight_df)[!,:weight][1]) for i in 1:num_courses))
hsg_appr_avg_constr = @constraint(mod_real,  hsg_wmean_appr == 0.0) 

#real_wvar_appr = @NLexpression(mod_real, (num_courses/(num_courses-1))*sum(((filter(row -> (row.c_idx == i-1) , cweight_df)[!,:weight][1])*(abs(c_sg_appr[i]))*(abs(c_sg_appr[i]))) for i in 1:num_courses)) # weighted variance
#real_appr_var_constr = @NLconstraint(mod_real,  real_wvar_appr == 1.0)


psg_wmean_appr = @expression(mod_real, sum(p_sg_appr[i]*(filter(row -> (row.p_idx == i-1) , pweight_df)[!,:weight][1]) for i in 1:num_plyrs))
psg_appr_avg_constr = @constraint(mod_real,  psg_wmean_appr == 0.0) 

real_pwvar_appr = @NLexpression(mod_real, (num_plyrs/(num_plyrs-1))*sum(((filter(row -> (row.p_idx == i-1) , pweight_df)[!,:weight][1])*(abs(p_sg_appr[i]))*(abs(p_sg_appr[i]))) for i in 1:num_plyrs)) # weighted variance
real_pappr_var_constr = @NLconstraint(mod_real,  real_pwvar_appr == 1.0)

dsr_appr_constr = @constraint(mod_real, [i=1:num_courses], 0 <= c_dsr_appr[i] <= 1)

sg_appr_apx_constr = @constraint(mod_real, [i=1:num_courses], filter(row -> (row.c_idx == i-1) , cweight_df)[!,:sg_appr_mean][1] -3 <= c_sg_appr[i] <= filter(row -> (row.c_idx == i-1) , cweight_df)[!,:sg_appr_mean][1] + 3) 
psg_appr_apx_constr = @constraint(mod_real, [i=1:num_plyrs], filter(row -> (row.p_idx == i-1) , pweight_df)[!,:sg_appr_mean][1]-3 <= p_sg_appr[i] <= filter(row -> (row.p_idx == i-1) , pweight_df)[!,:sg_appr_mean][1] +3)


#dsr_wmean_short = @expression(mod_real, sum(c_dsr_short[i]*(filter(row -> (row.c_idx == i-1) , cweight_df)[!,:weight][1]) for i in 1:num_courses))
#dsr_short_avg_constr = @constraint(mod_real,  dsr_wmean_short == 0.5) 

hsg_wmean_short = @expression(mod_real, sum(c_sg_short[i]*(filter(row -> (row.c_idx == i-1) , cweight_df)[!,:weight][1]) for i in 1:num_courses))
hsg_short_avg_constr = @constraint(mod_real,  hsg_wmean_short == 0.0) 

#real_wvar_short = @NLexpression(mod_real, (num_courses/(num_courses-1))*sum(((filter(row -> (row.c_idx == i-1) , cweight_df)[!,:weight][1])*(abs(c_sg_short[i]))*(abs(c_sg_short[i]))) for i in 1:num_courses)) # weighted variance
#real_short_var_constr = @NLconstraint(mod_real,  real_wvar_short == 1.0)


psg_wmean_short = @expression(mod_real, sum(p_sg_short[i]*(filter(row -> (row.p_idx == i-1) , pweight_df)[!,:weight][1]) for i in 1:num_plyrs))
psg_short_avg_constr = @constraint(mod_real,  psg_wmean_short == 0.0) 

real_pwvar_short = @NLexpression(mod_real, (num_plyrs/(num_plyrs-1))*sum(((filter(row -> (row.p_idx == i-1) , pweight_df)[!,:weight][1])*(abs(p_sg_short[i]))*(abs(p_sg_short[i]))) for i in 1:num_plyrs)) # weighted variance
real_pshort_var_constr = @NLconstraint(mod_real,  real_pwvar_short == 1.0)


dsr_short_constr = @constraint(mod_real, [i=1:num_courses], 0 <= c_dsr_short[i] <= 1) 

sg_short_apx_constr = @constraint(mod_real, [i=1:num_courses], filter(row -> (row.c_idx == i-1) , cweight_df)[!,:sg_short_mean][1] -3 <= c_sg_short[i] <= filter(row -> (row.c_idx == i-1) , cweight_df)[!,:sg_short_mean][1] +3) 
psg_short_apx_constr = @constraint(mod_real, [i=1:num_plyrs], filter(row -> (row.p_idx == i-1) , pweight_df)[!,:sg_short_mean][1]-3 <= p_sg_short[i] <= filter(row -> (row.p_idx == i-1) , pweight_df)[!,:sg_short_mean][1]+3)

#dsr_wmean_putt = @expression(mod_real, sum(c_dsr_putt[i]*(filter(row -> (row.X_c_idx == i-1) , cweight_df)[!,:weight][1]) for i in 1:num_courses))
#dsr_putt_avg_constr = @constraint(mod_real,  dsr_wmean_putt == 0.5) 

hsg_wmean_putt = @expression(mod_real, sum(c_sg_putt[i]*(filter(row -> (row.c_idx == i-1) , cweight_df)[!,:weight][1]) for i in 1:num_courses))
hsg_putt_avg_constr = @constraint(mod_real,  hsg_wmean_putt == 0.0) 

#real_wvar_putt = @NLexpression(mod_real, (num_courses/(num_courses-1))*sum(((filter(row -> (row.c_idx == i-1) , cweight_df)[!,:weight][1])*(abs(c_sg_putt[i]))*(abs(c_sg_putt[i]))) for i in 1:num_courses)) # weighted variance
#real_putt_var_constr = @NLconstraint(mod_real,  real_wvar_putt == 1.0)


psg_wmean_putt = @expression(mod_real, sum(p_sg_putt[i]*(filter(row -> (row.p_idx == i-1) , pweight_df)[!,:weight][1]) for i in 1:num_plyrs))
psg_putt_avg_constr = @constraint(mod_real,  psg_wmean_putt == 0.0) 

real_pwvar_putt = @NLexpression(mod_real, (num_plyrs/(num_plyrs-1))*sum(((filter(row -> (row.p_idx == i-1) , pweight_df)[!,:weight][1])*(abs(p_sg_putt[i]))*(abs(p_sg_putt[i]))) for i in 1:num_plyrs)) # weighted variance
real_pputt_var_constr = @NLconstraint(mod_real,  real_pwvar_putt == 1.0)


dsr_putt_constr = @constraint(mod_real, [i=1:num_courses], 0 <= c_dsr_putt[i]<= 1 ) 

sg_putt_apx_constr = @constraint(mod_real, [i=1:num_courses], filter(row -> (row.c_idx == i-1) , cweight_df)[!,:sg_putt_mean][1] -3 <= c_sg_putt[i] <= filter(row -> (row.c_idx == i-1) , cweight_df)[!,:sg_putt_mean][1] + 3) 
psg_putt_apx_constr = @constraint(mod_real, [i=1:num_plyrs], filter(row -> (row.p_idx == i-1) , pweight_df)[!,:sg_putt_mean][1]-3 <= p_sg_putt[i] <= filter(row -> (row.p_idx == i-1) , pweight_df)[!,:sg_putt_mean][1]+3)


# p*d - c
@NLobjective(mod_real, Min, sum( (p_sg_drive_[i]*p_sg_drive_[i]*c_dsr_drive_[i]*c_dsr_drive_[i] - 2*p_sg_drive_[i]*c_dsr_drive_[i]*c_sg_drive_[i] + c_sg_drive_[i]*c_sg_drive_[i]- 2*p_sg_drive_[i]*c_dsr_drive_[i]*real_data[!,:SG_drive][i] + 2*c_sg_drive_[i]*real_data[!,:SG_drive][i] + real_data[!,:SG_drive][i]*real_data[!,:SG_drive][i]) + (p_sg_appr_[i]*p_sg_appr_[i]*c_dsr_appr_[i]*c_dsr_appr_[i] - 2*p_sg_appr_[i]*c_dsr_appr_[i]*c_sg_appr_[i] + c_sg_appr_[i]*c_sg_appr_[i]- 2*p_sg_appr_[i]*c_dsr_appr_[i]*real_data[!,:SG_appr][i] + 2*c_sg_appr_[i]*real_data[!,:SG_appr][i] + real_data[!,:SG_appr][i]*real_data[!,:SG_appr][i]) + (p_sg_short_[i]*p_sg_short_[i]*c_dsr_short_[i]*c_dsr_short_[i] - 2*p_sg_short_[i]*c_dsr_short_[i]*c_sg_short_[i] + c_sg_short_[i]*c_sg_short_[i]- 2*p_sg_short_[i]*c_dsr_short_[i]*real_data[!,:SG_short][i] + 2*c_sg_short_[i]*real_data[!,:SG_short][i] + real_data[!,:SG_short][i]*real_data[!,:SG_short][i]) + (p_sg_putt_[i]*p_sg_putt_[i]*c_dsr_putt_[i]*c_dsr_putt_[i] - 2*p_sg_putt_[i]*c_dsr_putt_[i]*c_sg_putt_[i] + c_sg_putt_[i]*c_sg_putt_[i]- 2*p_sg_putt_[i]*c_dsr_putt_[i]*real_data[!,:SG_putt][i] + 2*c_sg_putt_[i]*real_data[!,:SG_putt][i] + real_data[!,:SG_putt][i]*real_data[!,:SG_putt][i])  for i in 1:n_real)/(n_real))

status_real = optimize!(mod_real)

c_idx_df = DataFrame(idx = unique(real_data[!,:c_idx]), course_name = unique(real_data[!,:course_name]))
p_idx_df = DataFrame(idx = unique(real_data[!,:p_idx]), player = unique(real_data[!,:player]))

p_d_label = [value.(p_sg_drive)[i+1] for i in p_idx_df[!,:idx]]
c_d_label = [value.(c_sg_drive)[i+1] for i in c_idx_df[!,:idx]]
cd_d_label = [value.(c_dsr_drive)[i+1] for i in c_idx_df[!,:idx]]

p_a_label = [value.(p_sg_appr)[i+1] for i in p_idx_df[!,:idx]]
c_a_label = [value.(c_sg_appr)[i+1] for i in c_idx_df[!,:idx]]
cd_a_label = [value.(c_dsr_appr)[i+1] for i in c_idx_df[!,:idx]]

p_s_label = [value.(p_sg_short)[i+1] for i in p_idx_df[!,:idx]]
c_s_label = [value.(c_sg_short)[i+1] for i in c_idx_df[!,:idx]]
cd_s_label = [value.(c_dsr_short)[i+1] for i in c_idx_df[!,:idx]]

p_p_label = [value.(p_sg_putt)[i+1] for i in p_idx_df[!,:idx]]
c_p_label = [value.(c_sg_putt)[i+1] for i in c_idx_df[!,:idx]]
cd_p_label = [value.(c_dsr_putt)[i+1] for i in c_idx_df[!,:idx]]

rounds = [filter(row -> (row.p_idx == i) , pweight_df)[!,:weight][1]*n_real for i in p_idx_df[!,:idx]]
obj_val =  objective_value(mod_real)

c_sg_final_df = DataFrame(c_idx = c_idx_df[!,:idx], course_name = c_idx_df[!,:course_name], sg_drive = c_d_label, sg_appr = c_a_label, sg_short = c_s_label, sg_putt = c_p_label)
c_dsr_final_df = DataFrame(c_idx = c_idx_df[!,:idx], course_name = c_idx_df[!,:course_name], dsr_drive = cd_d_label, dsr_appr = cd_a_label, dsr_short = cd_s_label, dsr_putt = cd_p_label)

p_final_df = DataFrame(p_idx = p_idx_df[!,:idx], player = p_idx_df[!,:player], sg_drive = p_d_label,sg_appr = p_a_label, sg_short = p_s_label, sg_putt = p_p_label, num_rounds = rounds)

obj = DataFrame(obj = obj_val)


CSV.write("C:/Users/Luke/OneDrive/Research/sg_data/realv2_2018_plyr_.csv", p_final_df)
CSV.write("C:/Users/Luke/OneDrive/Research/sg_data/realv2_2018_csg_sg_.csv", c_sg_final_df)
CSV.write("C:/Users/Luke/OneDrive/Research/sg_data/realv2_2018_c_sg_.csv", c_dsr_final_df)
CSV.write("C:/Users/Luke/OneDrive/Research/sg_data/realv2_2018_obj_.csv", obj)

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     4180
Number of nonzeros in inequality constraint Jacobian.:     4624
Number of nonzeros in Lagrangian Hessian.............:    43028

Total number of variables............................:     2312
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       12
Total number of inequality constraints...............:     4624
        inequality constraints with only lower bounds:     2312
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:     2312

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

"C:/Users/Luke/OneDrive/Research/sg_data/realv2_2018_obj_.csv"